## 6. Cluster_DatosFull

### Objetivo

Generar cluster's de estaciones en base sólo a la variable de demanda histórica

### Descripción General de notebook

    1. Carga de datos base proveniente de etapa de preparación
    2. Clusterización final en base a k=5

## 1. Carga de datos de entrada

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import sklearn.cluster as cluster
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
import sklearn.cluster as cluster
import sklearn.metrics as metrics
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

%matplotlib inline

pd.set_option('display.max_rows', 5)

In [2]:
df_FinalCluster_Original = pd.read_csv('../../data/df_FinalCluster_2017_2019.csv',
                              dtype={'Distrito':'category','Plazas':'category','Barrio':'category'})

df_FinalCluster_Original.info()
#df_FinalCluster

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   ESTACION             172 non-null    int64   
 1   DEMANDA_RATIO        172 non-null    float64 
 2   DMNDA_201703         172 non-null    float64 
 3   DMNDA_201704         172 non-null    float64 
 4   DMNDA_201705         172 non-null    int64   
 5   DMNDA_201706         172 non-null    int64   
 6   DMNDA_201707         172 non-null    int64   
 7   DMNDA_201708         172 non-null    int64   
 8   DMNDA_201709         172 non-null    int64   
 9   DMNDA_201710         172 non-null    int64   
 10  DMNDA_201711         172 non-null    int64   
 11  DMNDA_201712         172 non-null    float64 
 12  DMNDA_201801         172 non-null    float64 
 13  DMNDA_201802         172 non-null    int64   
 14  DMNDA_201803         172 non-null    int64   
 15  DMNDA_201804         17

### Selección de Variables DEMANDA Mensual

In [3]:
df_FinalCluster = df_FinalCluster_Original.copy()

df_FinalCluster = df_FinalCluster.drop(columns='ESTACION')

df_FinalCluster = df_FinalCluster[[
    'DMNDA_201703', 'DMNDA_201704', 'DMNDA_201705', 'DMNDA_201706', 
    'DMNDA_201707', 'DMNDA_201708', 'DMNDA_201709', 'DMNDA_201710', 'DMNDA_201711', 'DMNDA_201712',
    'DMNDA_201801', 'DMNDA_201802', 'DMNDA_201803', 'DMNDA_201804', 'DMNDA_201805', 'DMNDA_201806', 
    'DMNDA_201807', 'DMNDA_201808', 'DMNDA_201809', 'DMNDA_201810', 'DMNDA_201811', 'DMNDA_201812',
    'DMNDA_201901', 'DMNDA_201902', 'DMNDA_201903', 'DMNDA_201904', 'DMNDA_201905', 'DMNDA_201906', 
    'DMNDA_201907', 'DMNDA_201908', 'DMNDA_201909', 'DMNDA_201910', 'DMNDA_201911', 'DMNDA_201912']]

df_FinalCluster.info()

num_cols = list(df_FinalCluster._get_numeric_data().columns)
cat_cols = list(set(df_FinalCluster.columns) - set(num_cols))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DMNDA_201703  172 non-null    float64
 1   DMNDA_201704  172 non-null    float64
 2   DMNDA_201705  172 non-null    int64  
 3   DMNDA_201706  172 non-null    int64  
 4   DMNDA_201707  172 non-null    int64  
 5   DMNDA_201708  172 non-null    int64  
 6   DMNDA_201709  172 non-null    int64  
 7   DMNDA_201710  172 non-null    int64  
 8   DMNDA_201711  172 non-null    int64  
 9   DMNDA_201712  172 non-null    float64
 10  DMNDA_201801  172 non-null    float64
 11  DMNDA_201802  172 non-null    int64  
 12  DMNDA_201803  172 non-null    int64  
 13  DMNDA_201804  172 non-null    int64  
 14  DMNDA_201805  172 non-null    float64
 15  DMNDA_201806  172 non-null    float64
 16  DMNDA_201807  172 non-null    float64
 17  DMNDA_201808  172 non-null    float64
 18  DMNDA_201809  172 non-null    

In [4]:
df_FinalCluster

,DMNDA_201703,DMNDA_201704,DMNDA_201705,DMNDA_201706,DMNDA_201707,DMNDA_201708,DMNDA_201709,DMNDA_201710,DMNDA_201711,DMNDA_201712,...,DMNDA_201903,DMNDA_201904,DMNDA_201905,DMNDA_201906,DMNDA_201907,DMNDA_201908,DMNDA_201909,DMNDA_201910,DMNDA_201911,DMNDA_201912
0,7.0,2212.0,2942,2946,3229,2929,3685,3606,2879,2051.0,...,2618.0,2581.0,2948.0,2850.0,2824.0,2304.0,3133.0,3524.0,1306.0,29.0
1,3.0,1174.0,1579,1536,1597,1491,1924,2033,1543,1131.0,...,1266.0,1125.0,1819.0,1678.0,1499.0,1289.0,825.0,0.0,0.0,0.0
2,1.0,1338.0,1776,2398,2108,1862,2270,2193,1836,1484.0,...,1974.0,2052.0,2929.0,3243.0,2579.0,2016.0,2329.0,2246.0,1700.0,1288.0
3,1.0,1131.0,1154,1801,1522,1265,1873,1843,1514,1079.0,...,1142.0,1259.0,1876.0,2098.0,1282.0,1219.0,1784.0,1767.0,1260.0,663.0
4,2.0,1052.0,1318,1573,1453,1201,1597,1594,1335,1024.0,...,1222.0,1297.0,1840.0,1760.0,1433.0,1095.0,1451.0,1403.0,1116.0,739.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,0.0,1488.0,2031,2384,2312,2004,2643,2599,2109,1529.0,...,1887.0,1942.0,2768.0,2975.0,2392.0,1927.0,2142.0,1989.0,1412.0,1118.0
168,0.0,863.0,1127,1156,1059,846,1327,1438,1130,910.0,...,1131.0,971.0,1464.0,1471.0,1305.0,874.0,1289.0,1445.0,1085.0,832.0
169,0.0,785.0,1005,1115,978,716,1125,1150,991,777.0,...,951.0,836.0,1188.0,1397.0,1193.0,825.0,1267.0,1339.0,957.0,652.0
170,0.0,845.0,1046,1162,856,647,1190,1230,981,858.0,...,721.0,645.0,863.0,976.0,791.0,650.0,1025.0,1077.0,698.0,446.0


In [5]:
print(num_cols)
print(cat_cols)

['DMNDA_201703', 'DMNDA_201704', 'DMNDA_201705', 'DMNDA_201706', 'DMNDA_201707', 'DMNDA_201708', 'DMNDA_201709', 'DMNDA_201710', 'DMNDA_201711', 'DMNDA_201712', 'DMNDA_201801', 'DMNDA_201802', 'DMNDA_201803', 'DMNDA_201804', 'DMNDA_201805', 'DMNDA_201806', 'DMNDA_201807', 'DMNDA_201808', 'DMNDA_201809', 'DMNDA_201810', 'DMNDA_201811', 'DMNDA_201812', 'DMNDA_201901', 'DMNDA_201902', 'DMNDA_201903', 'DMNDA_201904', 'DMNDA_201905', 'DMNDA_201906', 'DMNDA_201907', 'DMNDA_201908', 'DMNDA_201909', 'DMNDA_201910', 'DMNDA_201911', 'DMNDA_201912']
[]


### Concatenación de DF: Numéricos y Categóricas

In [7]:
df_knn_final = df_FinalCluster[num_cols].copy()
df_knn_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DMNDA_201703  172 non-null    float64
 1   DMNDA_201704  172 non-null    float64
 2   DMNDA_201705  172 non-null    int64  
 3   DMNDA_201706  172 non-null    int64  
 4   DMNDA_201707  172 non-null    int64  
 5   DMNDA_201708  172 non-null    int64  
 6   DMNDA_201709  172 non-null    int64  
 7   DMNDA_201710  172 non-null    int64  
 8   DMNDA_201711  172 non-null    int64  
 9   DMNDA_201712  172 non-null    float64
 10  DMNDA_201801  172 non-null    float64
 11  DMNDA_201802  172 non-null    int64  
 12  DMNDA_201803  172 non-null    int64  
 13  DMNDA_201804  172 non-null    int64  
 14  DMNDA_201805  172 non-null    float64
 15  DMNDA_201806  172 non-null    float64
 16  DMNDA_201807  172 non-null    float64
 17  DMNDA_201808  172 non-null    float64
 18  DMNDA_201809  172 non-null    

## 2. Clusterización final en base a k=5

La clusterización se ejecuta para cada DataFrame según PCA siempre considerando k=5. 
La ejecución del algoritmo final considera:

    - Estandarización: MinMaxScaler()
    - Clusterización: KMeans según parámetros observados en siguiente celda

In [8]:
n_cluster = 5

scaler = MinMaxScaler()
kmeans = KMeans(
               n_clusters=n_cluster,
               init="k-means++",
               n_init=50,
               max_iter=500,
               random_state=200,)

pipeline = make_pipeline(scaler, kmeans)
pipeline

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kmeans',
                 KMeans(max_iter=500, n_clusters=5, n_init=50,
                        random_state=200))])

In [9]:
df_pred_knn_final_t = pipeline.fit_transform(df_knn_final)
df_pred_knn_final = pipeline.fit_predict(df_knn_final)


In [10]:
df_pred_knn_final_t

array([[0.97463155, 2.57840205, 1.4048747 , 1.87258605, 1.43424812],
       [1.79217262, 0.79564322, 3.00883193, 0.70922823, 1.15111951],
       [0.7097133 , 1.52451746, 1.94836771, 0.75109485, 0.2774712 ],
       [1.54622748, 0.66676391, 2.80593471, 0.27917903, 0.79331449],
       [1.65488919, 0.51905002, 2.92262681, 0.30725274, 0.89562817],
       [0.33326933, 2.36863846, 1.11324057, 1.5940918 , 1.00563228],
       [1.33129711, 0.87216049, 2.58437159, 0.2054958 , 0.58000744],
       [1.02488864, 1.15889864, 2.28610656, 0.39142792, 0.28627128],
       [0.74118421, 2.84142695, 0.68308377, 2.06634114, 1.474329  ],
       [1.17328187, 1.01140167, 2.43863326, 0.28964285, 0.47132404],
       [1.27468984, 0.96151629, 2.53252919, 0.34437972, 0.58199089],
       [0.36272727, 2.25643772, 1.2612054 , 1.48670578, 0.89941656],
       [1.99210215, 0.27841307, 3.25759154, 0.65337318, 1.2405243 ],
       [1.61993802, 0.97527671, 2.80666391, 0.68438366, 1.01251119],
       [0.91327424, 1.33072555, 2.

In [11]:
df_pred_knn_final

array([0, 3, 4, 3, 3, 0, 3, 4, 2, 3, 3, 0, 1, 3, 4, 3, 0, 3, 1, 1, 3, 0,
       3, 1, 4, 3, 1, 1, 0, 1, 0, 4, 2, 3, 0, 0, 3, 4, 0, 3, 3, 0, 4, 3,
       4, 2, 2, 0, 0, 3, 2, 3, 3, 4, 0, 4, 4, 3, 3, 0, 4, 2, 3, 4, 1, 1,
       1, 3, 4, 1, 0, 1, 3, 1, 4, 0, 4, 3, 4, 1, 3, 1, 3, 3, 3, 3, 0, 2,
       4, 0, 3, 2, 4, 4, 1, 3, 3, 4, 1, 1, 2, 0, 2, 0, 0, 2, 0, 0, 3, 1,
       1, 3, 1, 0, 1, 1, 0, 4, 3, 4, 3, 0, 1, 3, 1, 1, 1, 3, 0, 1, 1, 3,
       1, 1, 2, 1, 3, 4, 1, 3, 1, 1, 1, 4, 1, 1, 1, 1, 4, 0, 3, 4, 1, 3,
       0, 1, 3, 1, 1, 3, 1, 1, 3, 3, 3, 3, 1, 4, 1, 1, 1, 2])

In [12]:
df_final_Clusterizado = pd.concat([df_FinalCluster_Original, pd.DataFrame(df_pred_knn_final)], axis=1)

df_final_Clusterizado.rename(columns={0: 'CLUSTER'}, inplace=True)
df_final_Clusterizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 48 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   ESTACION             172 non-null    int64   
 1   DEMANDA_RATIO        172 non-null    float64 
 2   DMNDA_201703         172 non-null    float64 
 3   DMNDA_201704         172 non-null    float64 
 4   DMNDA_201705         172 non-null    int64   
 5   DMNDA_201706         172 non-null    int64   
 6   DMNDA_201707         172 non-null    int64   
 7   DMNDA_201708         172 non-null    int64   
 8   DMNDA_201709         172 non-null    int64   
 9   DMNDA_201710         172 non-null    int64   
 10  DMNDA_201711         172 non-null    int64   
 11  DMNDA_201712         172 non-null    float64 
 12  DMNDA_201801         172 non-null    float64 
 13  DMNDA_201802         172 non-null    int64   
 14  DMNDA_201803         172 non-null    int64   
 15  DMNDA_201804         17

In [13]:
df_final_Clusterizado[['ESTACION', 'CLUSTER']]

,ESTACION,CLUSTER
0,1,0
1,2,3
...,...,...
170,146,1
171,175,2


In [14]:
df_final_Clusterizado[['ESTACION', 'CLUSTER']].to_csv('../../data/Clusterizada_soloDemandaMensual.csv', index=False)